In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline
import joblib
from warnings import filterwarnings
import random
random.seed(0)
filterwarnings(action = "ignore")

In [2]:
# df = pd.read_csv("./telco_with_probabilties_new.csv")
df = pd.read_csv("/kaggle/input/1-estimating-probabilities/Data_with_probabilities.csv")

## 1. Data Exploration

In [3]:
df.columns

Index(['Customer ID', 'Gender', 'Age', 'Under 30', 'Senior Citizen', 'Married',
       'Dependents', 'Number of Dependents', 'Country', 'State',
       ...
       'Z42', 'Z43', 'Z44', 'Z45', 'Z50', 'Z51', 'Z52', 'Z53', 'Z54', 'Z55'],
      dtype='object', length=164)

In [4]:
df.shape

(7043, 164)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Columns: 164 entries, Customer ID to Z55
dtypes: bool(1), float64(116), int64(16), object(31)
memory usage: 8.8+ MB


### 1.1. By exploring each of the columns, the initial impression classifies the input data features into the following numerical, categorical and ordinal features

**Note**: analyzed the frequency of each feature using `df[f"{feature_name}"].value_counts()`

In [6]:
numerical_features = [
    'Age', 'Latitude', 'Longitude', 'Population', 'Tenure in Months', 'Avg Monthly Long Distance Charges',
    'Avg Monthly GB Download', 'Monthly Charge', 'Total Charges', 'Total Refunds', 'Total Extra Data Charges',
    'Total Long Distance Charges', 'Total Revenue', 'Churn Score', 'CLTV', 
    'Number of Dependents', 'Number of Referrals', 
    'p0', 'p1', 'p2', 'p3', 'p4', 'p5',
    'X00', 'X01', 'X02', 'X03', 'X04', 'X05', 'X10', 'X20', 'X30', 'X40', 'X50', 
    'Y00', 'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y10', 'Y20', 'Y30', 'Y40', 'Y50', 
    'Z00', 'Z01', 'Z02', 'Z03', 'Z04', 'Z05', 'Z10', 'Z20', 'Z30', 'Z40', 'Z50',
    'X11', 'X12', 'X13', 'X14', 'X15', 'X21', 'X22', 'X23', 'X24', 'X25', 'X31', 'X32', 'X33', 'X34', 'X35', 'X41', 'X42', 'X43', 'X44', 'X45', 'X51', 'X52', 'X53', 'X54', 'X55',
    'Y11', 'Y12', 'Y13', 'Y14', 'Y15', 'Y21', 'Y22', 'Y23', 'Y24', 'Y25', 'Y31', 'Y32', 'Y33', 'Y34', 'Y35', 'Y41', 'Y42', 'Y43', 'Y44', 'Y45', 'Y51', 'Y52', 'Y53', 'Y54', 'Y55', 
    'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z21', 'Z22', 'Z23', 'Z24', 'Z25', 'Z31', 'Z32', 'Z33', 'Z34', 'Z35', 'Z41', 'Z42', 'Z43', 'Z44', 'Z45', 'Z51', 'Z52', 'Z53', 'Z54', 'Z55',
    "Zip Code"
]

categorical_features = [
    'Customer ID', "Gender", "City", "State", "Country", "Quarter", "Churn Category", "Churn Reason", "Referred a Friend", "Internet Service",
    "Offer", "Internet Type", "Payment Method", "Customer Status", "Churn Label", "Married", "Dependents", "Phone Service", "Multiple Lines", 
    "Unlimited Data", "Online Security", "Online Backup", "Under 30", "Senior Citizen", "Premium Tech Support", "Streaming TV",
    "Streaming Movies", "Streaming Music", "Paperless Billing", "Device Protection Plan"
]

ordinal_features = ['Contract', 'Satisfaction Score']

print(len(set(numerical_features + categorical_features + ordinal_features)))

164


## 2. Data Wrangling

### 2.1. Dropping duplicates

In [7]:
df.duplicated().any()

False

### 2.2. Transforming flag features into numerical features

**observation**<br>
Treating Flag features as Numerical features is a standard practice in Machine Learning, specifically for tree-based regression models. So, the following flag features will be translated into numerical features
> 1. `Churn Label`
> 2. `Dependents`
> 3. `Device Protection Plan`
> 4. `Internet Service`
> 5. `Married`
> 6. `Multiple Lines`
> 7. `Online Backup`
> 8. `Online Security`
> 9. `Paperless Billing`
> 10. `Phone Service`
> 11. `Premium Tech Support`
> 12. `Referred a Friend`
> 13. `Senior Citizen`
> 14. `Streaming Movies`
> 15. `Streaming Music`
> 16. `Streaming TV`
> 17. `Under 30`
> 18. `Unlimited Data`

In [8]:
for feature in ['Churn Label', 'Dependents', 'Device Protection Plan', 'Internet Service', 
                'Married', 'Multiple Lines', 'Online Backup', 'Online Security', 
                'Paperless Billing', 'Phone Service', 'Premium Tech Support', 'Referred a Friend', 'Senior Citizen', 
                'Streaming Movies', 'Streaming Music', 'Streaming TV', 'Under 30', 'Unlimited Data']:
    if df[feature].isna().sum():
        print(feature)

So, none of the above features have missing values.<br>Transforming the above features into numerical features adhering to the standard practice

In [9]:
# finding the unique values from each of the features.
values = []
for feature in ['Churn Label', 'Dependents', 'Device Protection Plan', 'Internet Service', 
                'Married', 'Multiple Lines', 'Online Backup', 'Online Security', 
                'Paperless Billing', 'Phone Service', 'Premium Tech Support', 'Referred a Friend', 'Senior Citizen', 
                'Streaming Movies', 'Streaming Music', 'Streaming TV', 'Under 30', 'Unlimited Data']:
    values.extend(list(set(df[feature].unique())))
set(values)

{'No', 'Yes'}

In [10]:
for feature in ['Churn Label', 'Dependents', 'Device Protection Plan', 'Internet Service', 
                'Married', 'Multiple Lines', 'Online Backup', 'Online Security', 'Paperless Billing', 'Phone Service', 
                'Premium Tech Support', 'Referred a Friend', 'Senior Citizen', 
                'Streaming Movies', 'Streaming Music', 'Streaming TV', 'Under 30', 'Unlimited Data']:
    df[feature] = df[feature].str.lower().map({'yes': 1, 'no': 0})

### 2.2. Revised classification of features into categorical, numerical and ordinal

In [11]:
numerical_features = [
    'Age', 'Latitude', 'Longitude', 'Population', 'Tenure in Months', 'Avg Monthly Long Distance Charges',
    'Avg Monthly GB Download', 'Monthly Charge', 'Total Charges', 'Total Refunds', 'Total Extra Data Charges',
    'Total Long Distance Charges', 'Total Revenue', 'Churn Score', 'CLTV', 
    'Number of Dependents', 'Number of Referrals', 
    'p0', 'p1', 'p2', 'p3', 'p4', 'p5',
    'X00', 'X01', 'X02', 'X03', 'X04', 'X05', 'X10', 'X20', 'X30', 'X40', 'X50', 
    'Y00', 'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y10', 'Y20', 'Y30', 'Y40', 'Y50', 
    'Z00', 'Z01', 'Z02', 'Z03', 'Z04', 'Z05', 'Z10', 'Z20', 'Z30', 'Z40', 'Z50',
    'X11', 'X12', 'X13', 'X14', 'X15', 'X21', 'X22', 'X23', 'X24', 'X25', 'X31', 'X32', 'X33', 'X34', 'X35', 'X41', 'X42', 'X43', 'X44', 'X45', 'X51', 'X52', 'X53', 'X54', 'X55',
    'Y11', 'Y12', 'Y13', 'Y14', 'Y15', 'Y21', 'Y22', 'Y23', 'Y24', 'Y25', 'Y31', 'Y32', 'Y33', 'Y34', 'Y35', 'Y41', 'Y42', 'Y43', 'Y44', 'Y45', 'Y51', 'Y52', 'Y53', 'Y54', 'Y55', 
    'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z21', 'Z22', 'Z23', 'Z24', 'Z25', 'Z31', 'Z32', 'Z33', 'Z34', 'Z35', 'Z41', 'Z42', 'Z43', 'Z44', 'Z45', 'Z51', 'Z52', 'Z53', 'Z54', 'Z55',
    "Zip Code",
    'Churn Label', 'Dependents', 'Device Protection Plan', 'Internet Service',  'Married', 
    'Multiple Lines', 'Online Backup', 'Online Security', 'Paperless Billing',
    'Phone Service', 'Premium Tech Support', 'Referred a Friend', 
    'Senior Citizen', 'Streaming Movies', 'Streaming Music', 'Streaming TV', 'Under 30', 'Unlimited Data'
]

categorical_features = [
    'Customer ID', 'Internet Type', 'Country', 'Offer', 'Payment Method', 'Gender', 
    'Customer Status', 'State', 'Quarter', 'City', 'Churn Category', 'Churn Reason']

ordinal_features = ['Contract', 'Satisfaction Score']

print(len(numerical_features + categorical_features + ordinal_features))

164


### 2.3. Filling the missing entries

In [12]:
for i in df.columns:
    count_of_null_entries = df[i].isna().sum()
    if count_of_null_entries:
        print(i, count_of_null_entries)

Offer 3877
Internet Type 1526
Churn Category 5174
Churn Reason 5174


In [13]:
df['Offer'].fillna('missing').value_counts()

Offer
missing    3877
Offer B     824
Offer E     805
Offer D     602
Offer A     520
Offer C     415
Name: count, dtype: int64

In [14]:
df['Offer'] = df['Offer'].fillna('No Offer Accepted')

In [15]:
df['Internet Type'].fillna('missing').value_counts()

Internet Type
Fiber Optic    3035
DSL            1652
missing        1526
Cable           830
Name: count, dtype: int64

In [16]:
df['Internet Type'] = df['Internet Type'].fillna('Not Subscribed')

In [17]:
df['Churn Category'].fillna('missing').value_counts()

Churn Category
missing            5174
Competitor          841
Attitude            314
Dissatisfaction     303
Price               211
Other               200
Name: count, dtype: int64

In [18]:
df['Churn Category'] = df['Churn Category'].fillna('Not Churned')

In [19]:
df['Churn Reason'].fillna('missing').value_counts()

Churn Reason
missing                                      5174
Competitor had better devices                 313
Competitor made better offer                  311
Attitude of support person                    220
Don't know                                    130
Competitor offered more data                  117
Competitor offered higher download speeds     100
Attitude of service provider                   94
Price too high                                 78
Product dissatisfaction                        77
Network reliability                            72
Long distance charges                          64
Service dissatisfaction                        63
Moved                                          46
Extra data charges                             39
Limited range of services                      37
Poor expertise of online support               31
Lack of affordable download/upload speed       30
Lack of self-service on Website                29
Poor expertise of phone support      

In [20]:
df['Churn Reason'] = df['Churn Reason'].fillna('Not Churned')

In [21]:
for i in df.columns:
    if df[i].isnull().sum():
        print(i)

> So, there are no rows with missing entries

### 2.4. Dropping the useless features

In [22]:
features_to_be_inspected = ['Customer ID', 'State', 'Country', 'Quarter']

In [23]:
df['Customer ID'].values

array(['8779-QRDMV', '7495-OOKFY', '1658-BYGOY', ..., '2234-XADUH',
       '4801-JZAZL', '3186-AJIEK'], dtype=object)

In [24]:
len(df['Customer ID'].unique())

7043

> So, `Customer ID` is just an identifier and carries no additional information useful for predictions

In [25]:
df['State'].unique()

array(['California'], dtype=object)

> So, `State` feature has only a single entry through out the data and thus carries no new information

In [26]:
df['Country'].unique()

array(['United States'], dtype=object)

> So, `Country` feature has only a single entry through out the data and thus carries no new information

In [27]:
df['Quarter'].unique()

array(['Q3'], dtype=object)

> So, `Quarter` feature has only a single entry through out the data and thus carries no new information

In [28]:
features_to_be_inspected = ['Customer ID', 'State', 'Country', 'Quarter']
df = df.drop(features_to_be_inspected, axis=1)

### 2.5. Updating categorical_features

In [29]:
categorical_features = list(set(categorical_features) - set(features_to_be_inspected))

### 2.7. Renaming the columns to lower case

In [30]:
df.columns = df.columns.str.lower()
df.columns

Index(['gender', 'age', 'under 30', 'senior citizen', 'married', 'dependents',
       'number of dependents', 'city', 'zip code', 'latitude',
       ...
       'z42', 'z43', 'z44', 'z45', 'z50', 'z51', 'z52', 'z53', 'z54', 'z55'],
      dtype='object', length=160)

In [31]:
numerical_features = [i.lower() for i in numerical_features]
ordinal_features = [i.lower() for i in ordinal_features]
categorical_features = [i.lower() for i in categorical_features]
print(len(numerical_features + categorical_features + ordinal_features))
# result will be 160 as 4 features were removed.

160


### 2.8. Prefixing zip code values with label zipcode-

In [32]:
df['zip code']

0       90022
1       90063
2       90065
3       90303
4       90602
        ...  
7038    92285
7039    92301
7040    92304
7041    92305
7042    92308
Name: zip code, Length: 7043, dtype: int64

> zip code should not be interpreted as a numerical column as scaling of this feature would make it lose it's initial purpose. It should rather be interpreted as a categorical value.

In [33]:
df['zip code'] = df['zip code'].apply(lambda x: 'zipcode-'+str(x))
df['zip code']

0       zipcode-90022
1       zipcode-90063
2       zipcode-90065
3       zipcode-90303
4       zipcode-90602
            ...      
7038    zipcode-92285
7039    zipcode-92301
7040    zipcode-92304
7041    zipcode-92305
7042    zipcode-92308
Name: zip code, Length: 7043, dtype: object

In [34]:
numerical_features = list(set(numerical_features) - set(['zip code']))
categorical_features = list(set(categorical_features).union(set(['zip code'])))
print(len(numerical_features + categorical_features + ordinal_features))

160


### 2.9. Feature Engineering

In [35]:
# total monthly charges paid over entire tenure
df['total_monthly_charges'] = df['monthly charge']*df['tenure in months']

In [36]:
# total number of products subscribed by each client
df['count_of_products_subscribed'] = df['p1'] + df['p2'] + df['p3'] + df['p4'] + df['p5']

In [37]:
# Finding total monthly charges spent per product on an average
df['total_monthly_charges_per_product'] = df['total_monthly_charges']/df['count_of_products_subscribed']

In [38]:
# actual cltv based on tenure
df['tenure_based_cltv'] = df['cltv'] * df['tenure in months']

In [39]:
# cltv of a customer per product on an average
df['cltv_to_products_count_ratio'] = df['cltv']/df['count_of_products_subscribed']
# churn score associated with a customer per product on an average
df['churn_score_to_products_count_ratio'] = df['churn score']/df['count_of_products_subscribed']
# satisfaction score of a customer per product on an average
df['satisfaction_score_to_products_count_ratio'] = df['satisfaction score']/df['count_of_products_subscribed']

In [40]:
# finding the number of years the customer is associated with the company
df['count_of_years_associated'] = df['tenure in months']//12

In [41]:
# evaluating a customer's apetite to churn based on the number of years associated with the company
df['churn_apetite_basis_tenure_association'] = df['count_of_years_associated'].apply(
    lambda x: 5 if x<=1 else
              (4 if x==2 else
              (3 if x==3 else
              (2 if x==4 else
              (1 if x>4 else None))))
)

In [42]:
# evaluating a customer's apetite to churn based on the number of products subscribed
df['churn_apetite_basis_products_subscription_count'] = df['count_of_products_subscribed'].apply(
    lambda x: 5 if x<=1 else(
              4 if x==2 else(
              3 if x==3 else(
              2 if x==4 else(
              1 if x>=5 else None))))
)

In [43]:
# evaluating a customer's apetite to churn based on the satisfaction score
df['churn_apetite_basis_satisfaction_score'] = df['satisfaction score'].apply(
    lambda x: 5 if x==1 else(
                4 if x==2 else(
                3 if x==3 else(
                2 if x==4 else(
                1 if x==5 else None))))
)

In [44]:
# adding the above features to numerical features
numerical_features = numerical_features + ['total_monthly_charges',
                                           'count_of_products_subscribed',
                                           'total_monthly_charges_per_product',
                                           'tenure_based_cltv',
                                           'cltv_to_products_count_ratio',
                                           'churn_score_to_products_count_ratio',
                                           'satisfaction_score_to_products_count_ratio',
                                           'count_of_years_associated',
                                           'churn_apetite_basis_tenure_association',
                                           'churn_apetite_basis_products_subscription_count',
                                           'churn_apetite_basis_satisfaction_score']
print(len(numerical_features + categorical_features + ordinal_features))

171


## 3. Encode

### 3.1. Converting categorical_features into one-hot vectors

In [45]:
encoder = OneHotEncoder()
encoder = encoder.fit(df[categorical_features])
encoded_data = encoder.transform(df[categorical_features]).toarray()
encoded_data_df = pd.DataFrame(encoded_data, columns = encoder.get_feature_names_out())
df_new = pd.concat([df, encoded_data_df], axis=1).drop(categorical_features, axis=1)

In [46]:
df_new.shape

(7043, 2939)

In [47]:
df = df_new.copy()

### 3.2. Encoding Ordinal features

In [48]:
set(df['satisfaction score'].values)

{1, 2, 3, 4, 5}

> So, `satisfaction score` is already encoded

In [49]:
set(df['contract'].unique())

{'Month-to-Month', 'One Year', 'Two Year'}

In [50]:
df['contract'] = df['contract'].map({ 'Month-to-Month': 1,
                                      'One Year': 2,
                                      'Two Year': 3 })

In [51]:
df['contract'].unique()

array([1, 2, 3])

> encoded `contract` feature to numerical values

In [52]:
target_features = []
for i in range(6):
    for j in range(6):
        target_features.extend([f'x{i}{j}', f'y{i}{j}', f'z{i}{j}'])
target_features = sorted(list(set(target_features)))

## 4. Modelling

In [53]:
model_dict = {}
scaler_dict = {}

In [54]:
# transforming column names to lower case again as one-hot encoding results new features
df.columns = df.columns.str.lower()

In [55]:
# splitting rows of data into train and test datasets, so that test dataset has 20% of the data points
df_train, df_test = train_test_split(df, test_size = 0.2, random_state = 1)

In [56]:
X_train = df_train.drop(target_features, axis=1)

In [57]:
def return_model_and_scaler(target_col, input_list_of_estimators = None):
    global df_train, X_train
    # if input_list_of_estimators are not passed to the function, list_of_estimators will be used
    if input_list_of_estimators == None:
        input_list_of_estimators = np.arange(2, 11)
    #df_train[target_col] = np.round(df_train[target_col], 2)
    y_train = df_train[target_col]
    # following ensures that grid search begins by scaling the data and then finding the best randomforest regressor using 5-fold cross validation and hyperparameter tuning
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('rf', RandomForestRegressor(random_state = 1))
    ])
    param_grid = {'rf__n_estimators': input_list_of_estimators}
    # n_jobs = -1 indicates to use all the processors
    grid_search = GridSearchCV(estimator = pipeline, param_grid = param_grid, cv=5, n_jobs = -1)
    grid_search.fit(X_train, y_train)
    # retrieving the model and scaler after training
    model = grid_search.best_estimator_.named_steps['rf']
    scaler = grid_search.best_estimator_.named_steps['scaler']
    #printing target_column, training_error, test_error
    print("\n")
    # printing target_column
    print(f"Target_column = {target_col}")
    # finding and printing training error
    X_train_scaled = scaler.transform(X_train)
    y_train_pred = model.predict(X_train_scaled)
    mse_train = mean_squared_error(y_train, y_train_pred)
    mae_train = mean_absolute_error(y_train, y_train_pred)
    print(f"Mean squared error for train data = {mse_train}")
    print(f"Mean absolute error for train data = {mae_train}")
    # finding and printing test error
    X_test_scaled = scaler.transform(df_test.drop(target_features, axis=1))
    y_test = df_test[target_col]
    y_test_pred = model.predict(X_test_scaled)
    mse_test = mean_squared_error(y_test, y_test_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    print(f"Mean squared error for test data = {mse_test}")
    print(f"Mean absolute error for test data = {mae_test}")
    # returning the scaler and model
    return scaler, model    

In [58]:
for feature in target_features:
    scaler, model = return_model_and_scaler(feature)
    scaler_dict[feature] = scaler
    model_dict[feature] = model



Target_column = x00
Mean squared error for train data = 6.075443255255235e-05
Mean absolute error for train data = 0.005724178299376267
Mean squared error for test data = 0.00035139273938877277
Mean absolute error for test data = 0.014541313208266743


Target_column = x01
Mean squared error for train data = 5.883119451271592e-05
Mean absolute error for train data = 0.005654517995941327
Mean squared error for test data = 0.0003491862951932623
Mean absolute error for test data = 0.014602602124513031


Target_column = x02
Mean squared error for train data = 6.354107416947867e-05
Mean absolute error for train data = 0.005840957266368068
Mean squared error for test data = 0.000335690672911784
Mean absolute error for test data = 0.014313945496061546


Target_column = x03
Mean squared error for train data = 5.973736670129185e-05
Mean absolute error for train data = 0.005710867601988897
Mean squared error for test data = 0.00036849921521842674
Mean absolute error for test data = 0.0149642694

## 5. Saving

In [59]:
for feature in target_features:
    joblib.dump(model_dict[feature], f"/kaggle/working/model_{feature}.joblib")

In [60]:
for feature in target_features:
    joblib.dump(scaler_dict[feature], f"/kaggle/working/scaler_{feature}.joblib")

In [61]:
df_train.to_csv("/kaggle/working/df_train.csv", index=None)
df_test.to_csv("/kaggle/working/df_test.csv", index=None)

In [62]:
import zipfile
with zipfile.ZipFile("/kaggle/working/models.zip", "w") as zipf:
    for feature in target_features:
        zipf.write(f"/kaggle/working/model_{feature}.joblib")

In [63]:
with zipfile.ZipFile("/kaggle/working/scalers.zip", "w") as zipf:
    for feature in target_features:
        zipf.write(f"/kaggle/working/scaler_{feature}.joblib")